# 네이버 제목으로 다음 뉴스에서 검색하기

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import openpyxl
import time
from tqdm import tqdm
# 네이버 기사 제목, 발행날짜


def get_news(query):
    news_df = pd.DataFrame(columns=("Title", "Datetime"))
    idx = 0

    # url_query = quote(query)
    for i in (range(1, 11, 10)):
        url = 'https://search.naver.com/search.naver?where=news&sort=1&query={}&start={}'.format(
            query, i)
        res = requests.get(url)
        res.raise_for_status()    # 문제생기면 종료

        soup = BeautifulSoup(res.text, "html.parser")
        cont = soup.find_all('div', {'class': 'news_area'})

        for i in cont:
            title = i.find('a', {'class': 'news_tit'}).text
            datetime = i.find('span', {'class': 'info'}).get_text()

            news_df.loc[idx] = [title, datetime]
            idx += 1

    return news_df


# 검색
query = input('검색어 : ')
news_df = get_news(query)
# news_df

# 기사제목으로 다음에서 검색
idx = 0
dnews_df = pd.DataFrame(columns=("Title", "Datetime", "Url", "Article"))
for n_title in tqdm(news_df['Title']):
    url = 'https://search.daum.net/search?w=news&q=' + n_title
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cont = soup.find('div', {'class': 'cont_inner'})
    if cont == None:
        title = news_df['Title'][0]
        datetime = 'None'
        art_href = 'None'
        href_cont = 'None'
    else:
        title = cont.find('a', {'class': 'f_link_b'}).text
        datetime = cont.find('span', {'class': 'f_nb date'}).text

    # href가 <a class="f_link_b" href="http://v.media.daum.net/v/20210227100223206?f=o" 이렇게 나옴. 
    # => 다음뉴스페이지가 아니라 오리지널신문사 페이지로 넘어감
    # 뒤에 숫자만 따서 'https://news.v.daum.net/v/' 여기에 붙여줌
    # 숫자 스플릿
        art_url = cont.find('a', {'class': 'f_link_b'})['href']

        if 'cp' in art_url:
            art_href = art_url
            href_cont = 'None'
        else:
            art_href = 'https://news.v.daum.net/v/' + \
                art_url.split('/')[4].split('?')[0]
            # art_href에 requests
            href_res = requests.get(art_href)
            href_soup = BeautifulSoup(href_res.text, 'html.parser')
            href_cont = href_soup.select_one(
                'div#harmonyContainer').text.replace('\n', '').strip()
            href_cont

    # dataframe으로 만들어서 엑셀파일 생성

    dnews_df.loc[idx] = [title, datetime, art_href, href_cont]
    idx += 1
    time.sleep(1)

name = input("저장할 파일명을 입력하세요 :")
xlsx = ".xlsx"
filename = name+xlsx
dnews_df.to_excel('./'+filename, index=False, encoding='utf-8')

# 문제 1 : 다음뉴스로 넘어가는 링크가 없는 기사 = 오리지널 링크만 있는 기사 는 href가 다름
# 문제 2 : 네이버 기사가 다음 기사에 없는 경우

# 문제 1 처리 :
# 문제 2 처리 : soup.find('div',{'class':'cont_inner'}) 이 None인 경우 title만 저장.


검색어 : 도지코인


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.19s/it]


저장할 파일명을 입력하세요 :dozi


## 문제점
- 다음뉴스가 아니면 크롤링을 못함.